In [10]:
import chipwhisperer as cw
import matplotlib.pyplot as plt

try :
    scope.dis()
    target.dis()
except Exception as e:
    pass

n = 4
scope = cw.scope()
target = cw.target(scope)
scope.default_setup()
cw.program_target(scope, cw.programmers.STM32FProgrammer, "crt-CWNANO-true.hex")


In [6]:
print(bin(0x5b))
print(bin(0x57))

0b1011011
0b1010111


In [22]:
hex(114)

'0x72'

In [2]:
import time
import numpy as np
scope.adc.samples = 100000
def reset_target(scope): 
    scope.io.nrst = 'low'
    time.sleep(0.01)
    scope.io.nrst = 'high'
    time.sleep(0.01)

def cap_pass_trace(pass_guess):
    ret = ""
    reset_target(scope)
    num_char = target.in_waiting()
    while num_char > 0:
        ret += target.read(num_char, 16)
        time.sleep(0.01)
        num_char = target.in_waiting()

    scope.arm()
#     target.write(pass_guess)
    target.simpleserial_write('s', pass_guess)

    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')

    trace = scope.get_last_trace(as_int=True)
    return np.array(trace)

In [13]:
inp = bytearray([0x4,0x0,0x0,0x0])
trace = cap_pass_trace(inp)

AttributeError: 'NoneType' object has no attribute 'controlWrite'

In [8]:
r = target.simpleserial_read('r',16)

In [9]:
[hex(i) for i in r]

['0xd2',
 '0x59',
 '0xf2',
 '0x4a',
 '0xf0',
 '0xa8',
 '0xe3',
 '0x8',
 '0xe',
 '0x12',
 '0xf3',
 '0xb2',
 '0xe',
 '0x70',
 '0x6d',
 '0x5']

In [12]:
N = 1043347985599790750547972239032002691
e = 65537

pow(396536242503841570983745957578538360,e,N)

4

In [18]:
lis = ['0xd2','0x59',
 '0xf2',
 '0x4a',
 '0xf0',
 '0xa8',
 '0xe3',
 '0x8']
num = 0
for i,ele in enumerate(lis):
    num += (1<<(8*i))*int(ele,16)
print(num)
import math
p = math.gcd(num-396536242503841570983745957578538360+N,N)
print(p)

640541322019756498
1084024262488859977


In [5]:
len(bin(1084024262488859977))

62

In [6]:
(96247659919005988*108402426248885997)

10433479855997907405585632572350036

In [43]:
# %matplotlib tk
# # trace = np.array(cap_pass_trace(bytearray([])))
# plt.plot(trace)
# plt.show()
pattern = trace[25:35]#correct for begin mod_exp
pattern1 = trace[8780:8793]#correct mod_mul
pattern2 = trace[8748:8770]#correct end of for loop

In [30]:
from numpy import genfromtxt
# np.savetxt("mod_mul_pat.csv", pattern1, delimiter=",")
# pattern = genfromtxt('mod_mul_pat.csv', delimiter=',')

In [44]:
from tqdm.notebook import tqdm
def pattern_match(pat,trace):
    len_p = len(pat)

    l = []

    for i in tqdm(range(len(trace)-len_p)):
        temp = trace[i:i+len_p]
        cf = np.corrcoef(pat,temp)[0][1]
        l.append(cf)
    return l

In [45]:
pt = pattern_match(pattern,trace)
pt1 = pattern_match(pattern1,trace)
pt2 = pattern_match(pattern2,trace)

  0%|          | 0/99990 [00:00<?, ?it/s]

  0%|          | 0/99987 [00:00<?, ?it/s]

  0%|          | 0/99978 [00:00<?, ?it/s]

In [46]:
%matplotlib tk
from matplotlib.patches import Rectangle


fig, ax = plt.subplots(2,sharex=False)
ax[0].plot(trace)
count = 0
l = []
for i,val in enumerate(pt):
    if val>0.97:
        count+=1
        print(i,val)
        l.append(i)
        rect = Rectangle((i, 0), len(pattern),256,facecolor='orange')
        ax[0].add_patch(rect)
        
print(count)
count = 0
l1 = []
for i,val in enumerate(pt1):
    if val>0.97:
        count+=1
        print(i,val)
        l1.append(i)
        rect = Rectangle((i, 0), len(pattern1),256,facecolor='lime')
        ax[0].add_patch(rect)
        
print(count)
count = 0
l2 = []
for i,val in enumerate(pt2):
    if val>0.97:
        count+=1
        print(i,val)
        l2.append(i)
        rect = Rectangle((i, 0), len(pattern2),256,facecolor='salmon')
        ax[0].add_patch(rect)
        

        
ax[1].plot(trace)
print(count)
    
plt.show()

25 1.0
29022 0.9990911154406737
2
57 0.9981565111960725
4136 0.9972299460989558
8780 1.0
10045 0.99690500950414
11342 0.9967500124666805
14065 0.9996939978531566
16385 0.9974505070471889
19177 0.9987752144642327
20534 0.99643204353818
21808 0.9967728672436744
25227 0.999391819534944
26765 0.9966294132885509
29054 0.9984555509622831
32912 0.9961058186888769
38406 0.9993698007716826
40343 0.9958293947957315
42310 0.9979190355180378
44596 0.9985682586484195
47082 0.9968290595245868
48377 0.9996846496885504
50327 0.995597424107861
52813 0.9978472557569381
53523 0.9989841466158509
55258 0.996136000307962
24
8748 1.0
11308 0.998064860621536
14033 0.9974700224919076
19145 0.9991468967513886
21774 0.9976460977108866
25195 0.9971961772549004
28973 0.9970660832418116
38374 0.9992230806911474
42278 0.9985813649288604
47048 0.9988678013333698
48345 0.9987959540082129
52779 0.9981108981563543
53491 0.9970942601760842
56655 0.9967020854798585
14


In [47]:
def find_val(l_mm,l_f):
    pre = 0
    num = ''
    for ele in l_f:
        temp = np.array(l_mm)
        val = len(temp[(temp>pre)*(temp<ele)])
        if val==2:
            num += '1'
        else :
            num+= '0'
        pre = ele
    num = num[::-1]
    print(num)
    print(int(num,2))

In [48]:
l.append(100000)
l1_ = np.array(l1)
l1_ = l1_[l1_<l[1]]
l2_ = np.array(l2)
l2_ = l2_[l2_<l[1]]
find_val(l1_,l2_)
l1_ = np.array(l1)
l1_ = l1_[l1_>l[1]]
l2_ = np.array(l2)
l2_ = l2_[l2_>l[1]]
find_val(l1_,l2_)

1011011
91
1010111
87


In [84]:
folder = "Our/"
np.savetxt(folder+"trace.csv", trace, delimiter=",")

In [49]:
import math

r = 2
e=3
n=17947
dp = 91
# p = 137
# q = 131
# phi = (p-1)*(q-1)
# k = 23
p = math.gcd(pow(r,e*(dp),n)-r%n+n,n)
print(p)

137


In [50]:
q = n/p
print(q)

131.0


In [55]:
N = 1043347985599790750547972239032002691
e = 65537
print(hex(N))
print(hex(1017127147047859737827211364811314852))
pow(4,e,N)

0xc8f106fb6a670462b39d69a5ff2883
0xc3e43d789b5cb0c6f8df0af28296a4


186460282150391377095990707611478650